# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Code](#code)
* [Methodology](#methodology)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find the relationship of using different type of data for clustering the neigborhoods of Barcelona, Spain. We will try to use socioecnomic data of every neighborhood and we will also use data of the most common venues in every neigborhood. It will be interesting to analyze if the clusters are similar because Barcelona is a multicultural city and also has a lot of tourism. This fact makes that usually the neighborhoods more visited for tourists are very different from other neighborhoods less visited.

I will use my few data science knowledge to analyze if the idea that I have of big difference between neigborhoods in the city is real.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our analysis are:
* Most common venues in every Neighborhood.
* Socio echonomic data of every Neighborhood in the city of Barcelona.
* Geographical coordinates of every Neighborhood.


The location in the map of every neighborhood will be the coordinates of the center of the hood and I will use the geopy API.

Following data sources will be needed to extract/generate the required information:
* The location in the map of every neighborhood will be the coordinates of the center of the hood and I will use the **Geopy API**.
* number of venues and their type in every neighborhood will be obtained using **Foursquare API**
* coordinate of Barcelona center will be obtained using **Google Maps API geocoding** of well known Barcelona location (Plaça Catalunya)
* The socioeconomic data will be extracted from the local government of Barcelona Website, you can acces there with the following link. https://www.bcn.cat/estadistica/catala/dades/barris/index.html (be carefull the web is only available in Spanish)



## Code <a name="data"></a>

First of all we need to import all the necessary libraries and the neighborhoods and demographic information

In [326]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis


df=pd.read_csv("barcelona_demogràfica.csv")#import the table from the csv file

dataframe=pd.DataFrame()
dataframe['Borough']=df['Borough']
dataframe['Neighborhood']=df['Neighborhood']
dataframe


,Borough,Neighborhood
0,Ciutat Vella,el Raval
1,Ciutat Vella,el Barri Gòtic
2,Ciutat Vella,la Barceloneta
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera"
4,Eixample,el Fort Pienc
5,Eixample,la Sagrada Família
6,Eixample,la Dreta de l'Eixample
7,Eixample,l'Antiga Esquerra de l'Eixample
8,Eixample,la Nova Esquerra de l'Eixample
9,Eixample,Sant Antoni


In [327]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


This function gives the coordinates of every neighborhood in Barcelona

In [328]:

def coordinates(adress):
    geolocator = Nominatim(user_agent="barcelona_explorer")
    location = geolocator.geocode(adress)
    latitude = location.latitude
    longitude = location.longitude
    #print('The geograpical coordinate of '+adress+' are {}, {}.'.format(latitude, longitude))
    return latitude,longitude
    
adress = 'Sant Andreu de Palomar, Sant Andreu, Barcelona'
c=coordinates(adress)
c


(41.4367844, 2.1908641)

We make an iteration using the predefined coordinates function to get the coordinates for every neighborhood in Barcelona and we store all the coordinates in a dataframe.

In [329]:
lat=[]
long=[]
for i in range(len(dataframe)):
    print(i)
    cord=coordinates(dataframe['Neighborhood'][i]+', '+dataframe['Borough'][i]+', Barcelona')
    lat=np.append(lat,cord[0])
    long=np.append(long,cord[1])
dataframe['Latitude']=lat
dataframe['Longitude']=long
dataframe
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72


,Borough,Neighborhood,Latitude,Longitude
0,Ciutat Vella,el Raval,41.379518,2.168368
1,Ciutat Vella,el Barri Gòtic,41.383395,2.176912
2,Ciutat Vella,la Barceloneta,41.380653,2.189927
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",41.372251,2.177532
4,Eixample,el Fort Pienc,41.395925,2.182325
5,Eixample,la Sagrada Família,41.403479,2.174410
6,Eixample,la Dreta de l'Eixample,41.394124,2.166471
7,Eixample,l'Antiga Esquerra de l'Eixample,41.388765,2.156597
8,Eixample,la Nova Esquerra de l'Eixample,41.382816,2.149966
9,Eixample,Sant Antoni,41.386699,2.163828


In [330]:

import folium # map rendering library

In [331]:
neighborhoods=dataframe
address = 'Barcelona, Spain'

geolocator = Nominatim(user_agent="barcelona_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Barcelona are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Barcelona are 41.3828939, 2.1774322.


We create a map where we can see all the neigborhoods centers of coordinates in the city

In [332]:
# create map of New York using latitude and longitude values
map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  
    
map_barcelona

In [333]:
barcelona_data=neighborhoods

In [334]:
CLIENT_ID = 'OKWPQU252GMGEDMHAWFNDF0OUOYS4GUPAVKN3GA2MD1Z2YFZ' # your Foursquare ID
CLIENT_SECRET = 'J351TQKR4O1EB1RZHK2E4HVTE2CK1TEVT1BO1LM1I3NIZQSA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OKWPQU252GMGEDMHAWFNDF0OUOYS4GUPAVKN3GA2MD1Z2YFZ
CLIENT_SECRET:J351TQKR4O1EB1RZHK2E4HVTE2CK1TEVT1BO1LM1I3NIZQSA


In [335]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 700 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OKWPQU252GMGEDMHAWFNDF0OUOYS4GUPAVKN3GA2MD1Z2YFZ&client_secret=J351TQKR4O1EB1RZHK2E4HVTE2CK1TEVT1BO1LM1I3NIZQSA&v=20180605&ll=41.3828939,2.1774322&radius=700&limit=100'

I use the same method used in the previous lab of New York neighborhoods to get all the venues data for every neighborhood

In [336]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    
    return(nearby_venues)

In [337]:
# type your answer here

bcn_venues = getNearbyVenues(names=barcelona_data['Neighborhood'],
                                   latitudes=barcelona_data['Latitude'],
                                   longitudes=barcelona_data['Longitude']
                                  )

el Raval  
el Barri Gòtic  
la Barceloneta  
Sant Pere, Santa Caterina i la Ribera  
el Fort Pienc  
la Sagrada Família  
la Dreta de l'Eixample  
l'Antiga Esquerra de l'Eixample  
la Nova Esquerra de l'Eixample  
Sant Antoni  
el Poble Sec
la Marina del Prat Vermell
la Marina de Port  
la Font de la Guatlla  
Hostafrancs  
la Bordeta  
Sants - Badal  
Sants  
les Corts  
la Maternitat i Sant Ramon  
Pedralbes  
Vallvidrera el Tibidabo i les Planes  
Sarrià  
les Tres Torres  
Sant Gervasi - la Bonanova  
Sant Gervasi - Galvany  
el Putxet i el Farró  
Vallcarca i els Penitents  
el Coll  
la Salut  
la Vila de Gràcia  
el Camp d'en Grassot i Gràcia Nova  
el Baix Guinardó  
Can Baró  
el Guinardó  
la Font d'en Fargues  
el Carmel  
la Teixonera  
Sant Genís dels Agudells  
Montbau  
la Vall d'Hebron  
la Clota  
Horta  
Vilapicina i la Torre Llobeta  
Porta  
el Turó de la Peira  
Can Peguera  
la Guineueta  
Canyelles  
les Roquetes  
Verdun  
la Prosperitat  
la Trinitat Nova  
Tor

In [338]:
print(bcn_venues.shape)
bcn_venues.head()

(4447, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,el Raval,41.379518,2.168368,La Robadora,41.379500,2.170463,Gastropub
1,el Raval,41.379518,2.168368,Chulapio,41.379264,2.165905,Cocktail Bar
2,el Raval,41.379518,2.168368,Las Fernández,41.378183,2.166354,Mediterranean Restaurant
3,el Raval,41.379518,2.168368,Robadors 23,41.379581,2.170603,Jazz Club
4,el Raval,41.379518,2.168368,Cera 23,41.378947,2.166180,Spanish Restaurant


In [339]:
bcn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Baró de Viver,63,63,63,63,63,63
Can Baró,34,34,34,34,34,34
Can Peguera,57,57,57,57,57,57
Canyelles,10,10,10,10,10,10
Ciutat Meridiana,9,9,9,9,9,9
Diagonal Mar i el Front Marítim del Poblenou,60,60,60,60,60,60
Horta,61,61,61,61,61,61
Hostafrancs,71,71,71,71,71,71
Montbau,31,31,31,31,31,31


In [340]:
print('There are {} uniques categories.'.format(len(bcn_venues['Venue Category'].unique())))

There are 298 uniques categories.


In [341]:


# one hot encoding
bcn_onehot = pd.get_dummies(bcn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bcn_onehot['Neighborhood'] = bcn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bcn_onehot.columns[-1]] + list(bcn_onehot.columns[:-1])
bcn_onehot = bcn_onehot[fixed_columns]

bcn_onehot.head()



,Zoo,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,Neighborhood,Nightclub,Noodle House,Nudist Beach,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tram Station,Transportation Service,Tu

In [342]:
bcn_grouped = bcn_onehot.groupby('Neighborhood').mean().reset_index()
bcn_grouped

,Neighborhood,Zoo,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Residence Hall,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,Nightclub,Noodle House,Nudist Beach,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tram Station,Transportation Service,Tu

In [343]:


num_top_venues = 5

for hood in bcn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bcn_grouped[bcn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')



----Baró de Viver  ----
            venue  freq
0  Clothing Store  0.13
1    Burger Joint  0.06
2  Ice Cream Shop  0.05
3   Women's Store  0.05
4  Sandwich Place  0.05


----Can Baró  ----
                venue  freq
0               Plaza  0.12
1  Spanish Restaurant  0.12
2       Grocery Store  0.09
3    Tapas Restaurant  0.09
4        Dessert Shop  0.06


----Can Peguera  ----
                venue  freq
0    Tapas Restaurant  0.12
1       Grocery Store  0.09
2         Pizza Place  0.07
3         Supermarket  0.05
4  Spanish Restaurant  0.04


----Canyelles  ----
           venue  freq
0   Soccer Field   0.2
1         Market   0.2
2          Plaza   0.1
3  Grocery Store   0.1
4     Skate Park   0.1


----Ciutat Meridiana  ----
           venue  freq
0  Metro Station  0.22
1    Supermarket  0.11
2           Café  0.11
3  Train Station  0.11
4  Grocery Store  0.11


----Diagonal Mar i el Front Marítim del Poblenou  ----
                      venue  freq
0  Mediterranean Restaurant  0.12

              venue  freq
0  Tapas Restaurant  0.13
1       Pizza Place  0.07
2      Cocktail Bar  0.07
3           Theater  0.05
4               Bar  0.05


----el Poblenou  ----
                      venue  freq
0  Mediterranean Restaurant  0.09
1        Spanish Restaurant  0.06
2          Tapas Restaurant  0.04
3                Restaurant  0.04
4        Italian Restaurant  0.04


----el Putxet i el Farró  ----
                venue  freq
0              Bakery  0.11
1                Café  0.08
2               Hotel  0.05
3  Italian Restaurant  0.05
4         Pizza Place  0.04


----el Raval  ----
                      venue  freq
0              Cocktail Bar  0.09
1        Spanish Restaurant  0.08
2  Mediterranean Restaurant  0.07
3          Tapas Restaurant  0.05
4                       Bar  0.05


----el Turó de la Peira  ----
              venue  freq
0  Tapas Restaurant  0.11
1        Restaurant  0.05
2     Grocery Store  0.05
3             Plaza  0.05
4       Supermarket  0.05




In [344]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [345]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bcn_grouped['Neighborhood']

for ind in np.arange(bcn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bcn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baró de Viver,Clothing Store,Burger Joint,Sandwich Place,Ice Cream Shop,Women's Store,Spanish Restaurant,Italian Restaurant,Fast Food Restaurant,Tapas Restaurant,Shopping Mall
1,Can Baró,Spanish Restaurant,Plaza,Grocery Store,Tapas Restaurant,Soccer Field,Chinese Restaurant,Café,Dessert Shop,Breakfast Spot,Tennis Court
2,Can Peguera,Tapas Restaurant,Grocery Store,Pizza Place,Supermarket,Restaurant,Burger Joint,Pub,Breakfast Spot,Seafood Restaurant,Spanish Restaurant
3,Canyelles,Market,Soccer Field,Skate Park,Grocery Store,Plaza,Mediterranean Restaurant,Café,Metro Station,Farmers Market,Fast Food Restaurant
4,Ciutat Meridiana,Metro Station,Park,Grocery Store,Plaza,Supermarket,Train Station,Café,Mediterranean Restaurant,Fish Market,Farmers Market


## Methodology <a name="methodology"></a>

Purpose of this project was to identify differences in clustering Barcelona Neighborhoods using two different types of data. By getting the most common venues in the neighborhoods we can start clustering the hoods using the kmeans clustering algorithm which will classify the more similar neighborhoods in the city in the same cluster.

We are going to use the same procedure for the socioeconomic data clustering. In this case the data is properly prepared in a csv file and we will take the columns of data that are relevant for the analysis and the other columns will be dropped from the dataframe. Then we can start clustering the hoods using the kmeans clustering algorithm which will classify the more similar neighborhoods in the city in the same cluster. 

In [346]:
# set number of clusters
kclusters = 5

bcn_grouped_clustering = bcn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bcn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 3, 4, 4, 1, 3, 1, 0, 3], dtype=int32)

In [347]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baró de Viver,Clothing Store,Burger Joint,Sandwich Place,Ice Cream Shop,Women's Store,Spanish Restaurant,Italian Restaurant,Fast Food Restaurant,Tapas Restaurant,Shopping Mall
1,Can Baró,Spanish Restaurant,Plaza,Grocery Store,Tapas Restaurant,Soccer Field,Chinese Restaurant,Café,Dessert Shop,Breakfast Spot,Tennis Court
2,Can Peguera,Tapas Restaurant,Grocery Store,Pizza Place,Supermarket,Restaurant,Burger Joint,Pub,Breakfast Spot,Seafood Restaurant,Spanish Restaurant
3,Canyelles,Market,Soccer Field,Skate Park,Grocery Store,Plaza,Mediterranean Restaurant,Café,Metro Station,Farmers Market,Fast Food Restaurant
4,Ciutat Meridiana,Metro Station,Park,Grocery Store,Plaza,Supermarket,Train Station,Café,Mediterranean Restaurant,Fish Market,Farmers Market
5,Diagonal Mar i el Front Marítim del Poblenou,Mediterranean Restaurant,Beach Bar,Restaurant,Spanish Restaurant,Hotel,Italian Restaurant,Athletics & Sports,Seafood Restaurant,Beach,Café
6,Horta,Tapas Restaurant,Spanish Restaurant,Café,Bakery,Park,Burger Joint,Supermarket,Mediterranean Restaurant,Pizza Place,Pub
7,Hostafrancs,Hotel,Tapas Restaurant,Bar,Plaza,Café,Wine Bar,Mediterranean Restaurant,Bakery,Spanish Restaurant,Gym
8,Montbau,Restaurant,Plaza,Park,Spanish Restaurant,Metro Station,Grocery Store,Café,Gym,Tennis Court,Tapas Restaurant
9,Navas,Supermarket,Grocery Store,Bakery,Spanish Restaurant,Restaurant,Tapas Restaurant,Gym / Fitness Center,Bar,Café,Japanese Restaurant


In [348]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bcn_merged = barcelona_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bcn_merged = bcn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bcn_merged # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ciutat Vella,el Raval,41.379518,2.168368,1,Cocktail Bar,Spanish Restaurant,Mediterranean Restaurant,Bar,Tapas Restaurant,Pizza Place,Hotel,Bookstore,Seafood Restaurant,Coffee Shop
1,Ciutat Vella,el Barri Gòtic,41.383395,2.176912,1,Tapas Restaurant,Spanish Restaurant,Hotel,Ice Cream Shop,Plaza,Cocktail Bar,Italian Restaurant,Wine Bar,Bar,Mediterranean Restaurant
2,Ciutat Vella,la Barceloneta,41.380653,2.189927,3,Tapas Restaurant,Paella Restaurant,Mediterranean Restaurant,Burger Joint,Seafood Restaurant,Beach,Bar,Restaurant,Spanish Restaurant,Board Shop
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",41.372251,2.177532,1,Hotel,Spanish Restaurant,Mediterranean Restaurant,Hostel,Park,Boat or Ferry,Garden,Mexican Restaurant,Italian Restaurant,Seafood Restaurant
4,Eixample,el Fort Pienc,41.395925,2.182325,1,Hotel,Coffee Shop,Restaurant,Bistro,Hostel,Mediterranean Restaurant,Spanish Restaurant,Bakery,Burger Joint,Theater
5,Eixample,la Sagrada Família,41.403479,2.174410,1,Hotel,Restaurant,Burger Joint,Italian Restaurant,Pizza Place,Café,Tapas Restaurant,Latin American Restaurant,Steakhouse,Plaza
6,Eixample,la Dreta de l'Eixample,41.394124,2.166471,1,Hotel,Hostel,Tapas Restaurant,Boutique,Bakery,Bookstore,Mediterranean Restaurant,Wine Bar,Sporting Goods Shop,Cosmetics Shop
7,Eixample,l'Antiga Esquerra de l'Eixample,41.388765,2.156597,1,Hotel,Spanish Restaurant,Cocktail Bar,Tapas Restaurant,Restaurant,Mediterranean Restaurant,Japanese Restaurant,Café,Sandwich Place,Coffee Shop
8,Eixample,la Nova Esquerra de l'Eixample,41.382816,2.149966,1,Hotel,Café,Tapas Restaurant,Mediterranean Restaurant,Gym,Japanese Restaurant,Restaurant,Coffee Shop,Bar,Bakery
9,Eixample,Sant Antoni,41.386699,2.163828,1,Hotel,Bookstore,Tapas Restaurant,Mediterranean Restaurant,Cocktail Bar,Sandwich Place,Hostel,Plaza,Coffee Shop,Cosmetics Shop


In [349]:
bcn_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ciutat Vella,el Raval,41.379518,2.168368,1,Cocktail Bar,Spanish Restaurant,Mediterranean Restaurant,Bar,Tapas Restaurant,Pizza Place,Hotel,Bookstore,Seafood Restaurant,Coffee Shop
1,Ciutat Vella,el Barri Gòtic,41.383395,2.176912,1,Tapas Restaurant,Spanish Restaurant,Hotel,Ice Cream Shop,Plaza,Cocktail Bar,Italian Restaurant,Wine Bar,Bar,Mediterranean Restaurant
2,Ciutat Vella,la Barceloneta,41.380653,2.189927,3,Tapas Restaurant,Paella Restaurant,Mediterranean Restaurant,Burger Joint,Seafood Restaurant,Beach,Bar,Restaurant,Spanish Restaurant,Board Shop
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",41.372251,2.177532,1,Hotel,Spanish Restaurant,Mediterranean Restaurant,Hostel,Park,Boat or Ferry,Garden,Mexican Restaurant,Italian Restaurant,Seafood Restaurant
4,Eixample,el Fort Pienc,41.395925,2.182325,1,Hotel,Coffee Shop,Restaurant,Bistro,Hostel,Mediterranean Restaurant,Spanish Restaurant,Bakery,Burger Joint,Theater
5,Eixample,la Sagrada Família,41.403479,2.174410,1,Hotel,Restaurant,Burger Joint,Italian Restaurant,Pizza Place,Café,Tapas Restaurant,Latin American Restaurant,Steakhouse,Plaza
6,Eixample,la Dreta de l'Eixample,41.394124,2.166471,1,Hotel,Hostel,Tapas Restaurant,Boutique,Bakery,Bookstore,Mediterranean Restaurant,Wine Bar,Sporting Goods Shop,Cosmetics Shop
7,Eixample,l'Antiga Esquerra de l'Eixample,41.388765,2.156597,1,Hotel,Spanish Restaurant,Cocktail Bar,Tapas Restaurant,Restaurant,Mediterranean Restaurant,Japanese Restaurant,Café,Sandwich Place,Coffee Shop
8,Eixample,la Nova Esquerra de l'Eixample,41.382816,2.149966,1,Hotel,Café,Tapas Restaurant,Mediterranean Restaurant,Gym,Japanese Restaurant,Restaurant,Coffee Shop,Bar,Bakery
9,Eixample,Sant Antoni,41.386699,2.163828,1,Hotel,Bookstore,Tapas Restaurant,Mediterranean Restaurant,Cocktail Bar,Sandwich Place,Hostel,Plaza,Coffee Shop,Cosmetics Shop


Now we get the map of neighborhoods once is clustered by venues data

In [350]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bcn_merged['Latitude'], bcn_merged['Longitude'], bcn_merged['Neighborhood'], bcn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



In [351]:
bcn_merged.loc[bcn_merged['Cluster Labels'] == 0, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,la Marina del Prat Vermell,Plaza,Mediterranean Restaurant,Auto Garage,Furniture / Home Store,Supermarket,Soccer Field,Food,Metro Station,Tennis Court,Flea Market
21,Vallvidrera el Tibidabo i les Planes,Hostel,Train Station,Convenience Store,Restaurant,Farmers Market,Building,National Park,Soccer Field,Spanish Restaurant,Tourist Information Center
27,Vallcarca i els Penitents,Café,Spanish Restaurant,Bakery,Grocery Store,Park,Plaza,Supermarket,Pizza Place,Hotel,Burger Joint
28,el Coll,Park,Scenic Lookout,Mountain,Plaza,Hotel,Mediterranean Restaurant,Spanish Restaurant,Grocery Store,Café,Tapas Restaurant
33,Can Baró,Spanish Restaurant,Plaza,Grocery Store,Tapas Restaurant,Soccer Field,Chinese Restaurant,Café,Dessert Shop,Breakfast Spot,Tennis Court
36,el Carmel,Grocery Store,Plaza,Gym,Food Court,Food & Drink Shop,Supermarket,Café,Metro Station,Fast Food Restaurant,Mediterranean Restaurant
37,la Teixonera,Grocery Store,Spanish Restaurant,Bakery,Gym,Soccer Field,Snack Place,Plaza,Food Court,Gas Station,Metro Station
38,Sant Genís dels Agudells,Bakery,Argentinian Restaurant,Sports Bar,Tapas Restaurant,Tennis Court,Farmers Market,Basketball Court,Grocery Store,Restaurant,Spanish Restaurant
39,Montbau,Restaurant,Plaza,Park,Spanish Restaurant,Metro Station,Grocery Store,Café,Gym,Tennis Court,Tapas Restaurant
40,la Vall d'Hebron,Grocery Store,Spanish Restaurant,Tapas Restaurant,Tennis Court,Restaurant,Metro Station,Gas Station,Farmers Market,Argentinian Restaurant,Bakery


In [352]:
bcn_merged.loc[bcn_merged['Cluster Labels'] == 1, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,el Raval,Cocktail Bar,Spanish Restaurant,Mediterranean Restaurant,Bar,Tapas Restaurant,Pizza Place,Hotel,Bookstore,Seafood Restaurant,Coffee Shop
1,el Barri Gòtic,Tapas Restaurant,Spanish Restaurant,Hotel,Ice Cream Shop,Plaza,Cocktail Bar,Italian Restaurant,Wine Bar,Bar,Mediterranean Restaurant
3,"Sant Pere, Santa Caterina i la Ribera",Hotel,Spanish Restaurant,Mediterranean Restaurant,Hostel,Park,Boat or Ferry,Garden,Mexican Restaurant,Italian Restaurant,Seafood Restaurant
4,el Fort Pienc,Hotel,Coffee Shop,Restaurant,Bistro,Hostel,Mediterranean Restaurant,Spanish Restaurant,Bakery,Burger Joint,Theater
5,la Sagrada Família,Hotel,Restaurant,Burger Joint,Italian Restaurant,Pizza Place,Café,Tapas Restaurant,Latin American Restaurant,Steakhouse,Plaza
6,la Dreta de l'Eixample,Hotel,Hostel,Tapas Restaurant,Boutique,Bakery,Bookstore,Mediterranean Restaurant,Wine Bar,Sporting Goods Shop,Cosmetics Shop
7,l'Antiga Esquerra de l'Eixample,Hotel,Spanish Restaurant,Cocktail Bar,Tapas Restaurant,Restaurant,Mediterranean Restaurant,Japanese Restaurant,Café,Sandwich Place,Coffee Shop
8,la Nova Esquerra de l'Eixample,Hotel,Café,Tapas Restaurant,Mediterranean Restaurant,Gym,Japanese Restaurant,Restaurant,Coffee Shop,Bar,Bakery
9,Sant Antoni,Hotel,Bookstore,Tapas Restaurant,Mediterranean Restaurant,Cocktail Bar,Sandwich Place,Hostel,Plaza,Coffee Shop,Cosmetics Shop
12,la Marina de Port,Clothing Store,Restaurant,Café,Spanish Restaurant,Supermarket,Coffee Shop,Hotel,Bakery,Sporting Goods Shop,Mediterranean Restaurant


In [353]:
bcn_merged.loc[bcn_merged['Cluster Labels'] == 2, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,la Trinitat Vella,Park,Metro Station,Spanish Restaurant,Tapas Restaurant,Soccer Field,Yoga Studio,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market


In [354]:
bcn_merged.loc[bcn_merged['Cluster Labels'] == 3, bcn_merged.columns[[1] + list(range(5, bcn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,la Barceloneta,Tapas Restaurant,Paella Restaurant,Mediterranean Restaurant,Burger Joint,Seafood Restaurant,Beach,Bar,Restaurant,Spanish Restaurant,Board Shop
10,el Poble Sec,Tapas Restaurant,Pizza Place,Cocktail Bar,Theater,Bar,Mediterranean Restaurant,Café,Spanish Restaurant,Park,Hotel
15,la Bordeta,Bar,Supermarket,Hotel,Restaurant,Café,Grocery Store,Park,Pizza Place,Tapas Restaurant,Japanese Restaurant
16,Sants - Badal,Pizza Place,Bakery,Tapas Restaurant,Café,Bar,Hostel,Supermarket,Grocery Store,Gym / Fitness Center,Ice Cream Shop
29,la Salut,Bar,Grocery Store,Plaza,Restaurant,Sandwich Place,Mediterranean Restaurant,Supermarket,Park,Tapas Restaurant,Spanish Restaurant
30,la Vila de Gràcia,Tapas Restaurant,Mediterranean Restaurant,Pizza Place,Cocktail Bar,Plaza,Bakery,Bar,Restaurant,Ice Cream Shop,Indie Movie Theater
31,el Camp d'en Grassot i Gràcia Nova,Tapas Restaurant,Restaurant,Indie Movie Theater,Hotel,Bistro,Grocery Store,Steakhouse,Vegetarian / Vegan Restaurant,Bar,Brewery
32,el Baix Guinardó,Bakery,Supermarket,Bar,Tapas Restaurant,Grocery Store,Hotel,Spanish Restaurant,Restaurant,Gym,Japanese Restaurant
34,el Guinardó,Gym,Tapas Restaurant,Grocery Store,Pharmacy,Pizza Place,Hotel,Plaza,Breakfast Spot,Restaurant,Spanish Restaurant
35,la Font d'en Fargues,Restaurant,Spanish Restaurant,Supermarket,Pub,Food & Drink Shop,Tapas Restaurant,Italian Restaurant,Scenic Lookout,Café,Gym


### Now we want to cluster Barcelona using social and demographic data of the Neighborhoods and we will compare how the clusters change

In [355]:
df=pd.read_csv("barcelona_demogràfica.csv")#import the table from the csv file


#we replace the commas to stops to convert strings to float number
columns=['RFD Index','% born Catalonia','% born Spain','% born Foreign','Average house occupation','% Unemployment','Lifespan','Average rental prices (€/month)']
for e in columns:
    df[e] = df[e].str.replace(',', '.').astype(float)

df=df.drop(['Borough','nascuts cat','nascuts espanya','nascuts estranger','població total','Population','N.º house','Average house occupation','Lifespan'] ,axis=1)

df

,Neighborhood,RFD Index,% born Catalonia,% born Spain,% born Foreign,% Unemployment,(€/m²) House prices,Average rental prices (€/month)
0,el Raval,71.2,37.65,12.27,50.08,9.52,4029,748.6
1,el Barri Gòtic,106.1,39.16,21.38,39.46,6.97,4884,973.3
2,la Barceloneta,79.6,54.43,13.45,32.11,9.19,5165,745.8
3,"Sant Pere, Santa Caterina i la Ribera",99.4,46.91,16.81,36.28,8.03,5152,916.7
4,el Fort Pienc,106.5,62.04,17.92,20.04,6.01,4107,900.4
5,la Sagrada Família,101.8,65.28,17.56,17.17,6.08,4209,876.2
6,la Dreta de l'Eixample,175.9,64.20,14.85,20.95,4.64,6332,1195.2
7,l'Antiga Esquerra de l'Eixample,137.2,66.32,14.25,19.44,5.25,5091,1060.0
8,la Nova Esquerra de l'Eixample,110.2,62.19,17.01,20.80,5.83,4465,943.2
9,Sant Antoni,104.2,62.85,16.64,20.51,6.36,4591,900.5


In [356]:
bcn_grouped=df
# set number of clusters
kclusters = 4

bcn_grouped_clustering = bcn_grouped.drop(['Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bcn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_


array([1, 1, 3, 3, 1, 1, 3, 3, 1, 1, 1, 1, 0, 2, 1, 2, 2, 2, 1, 1, 3, 1,
       3, 3, 3, 3, 1, 2, 1, 2, 1, 1, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 2, 2, 1,
       3, 1, 3, 0, 2, 0, 0], dtype=int32)

In [357]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bcn_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bcn_merged = bcn_merged.join(dataframe.set_index('Neighborhood'), on='Neighborhood')
bcn_merged['Cluster Labels'] = kmeans.labels_

bcn_merged # check the last columns!

,Neighborhood,RFD Index,% born Catalonia,% born Spain,% born Foreign,% Unemployment,(€/m²) House prices,Average rental prices (€/month),Borough,Latitude,Longitude,Cluster Labels
0,el Raval,71.2,37.65,12.27,50.08,9.52,4029,748.6,Ciutat Vella,41.379518,2.168368,1
1,el Barri Gòtic,106.1,39.16,21.38,39.46,6.97,4884,973.3,Ciutat Vella,41.383395,2.176912,1
2,la Barceloneta,79.6,54.43,13.45,32.11,9.19,5165,745.8,Ciutat Vella,41.380653,2.189927,3
3,"Sant Pere, Santa Caterina i la Ribera",99.4,46.91,16.81,36.28,8.03,5152,916.7,Ciutat Vella,41.372251,2.177532,3
4,el Fort Pienc,106.5,62.04,17.92,20.04,6.01,4107,900.4,Eixample,41.395925,2.182325,1
5,la Sagrada Família,101.8,65.28,17.56,17.17,6.08,4209,876.2,Eixample,41.403479,2.174410,1
6,la Dreta de l'Eixample,175.9,64.20,14.85,20.95,4.64,6332,1195.2,Eixample,41.394124,2.166471,3
7,l'Antiga Esquerra de l'Eixample,137.2,66.32,14.25,19.44,5.25,5091,1060.0,Eixample,41.388765,2.156597,3
8,la Nova Esquerra de l'Eixample,110.2,62.19,17.01,20.80,5.83,4465,943.2,Eixample,41.382816,2.149966,1
9,Sant Antoni,104.2,62.85,16.64,20.51,6.36,4591,900.5,Eixample,41.386699,2.163828,1


In [358]:
bcn_merged

,Neighborhood,RFD Index,% born Catalonia,% born Spain,% born Foreign,% Unemployment,(€/m²) House prices,Average rental prices (€/month),Borough,Latitude,Longitude,Cluster Labels
0,el Raval,71.2,37.65,12.27,50.08,9.52,4029,748.6,Ciutat Vella,41.379518,2.168368,1
1,el Barri Gòtic,106.1,39.16,21.38,39.46,6.97,4884,973.3,Ciutat Vella,41.383395,2.176912,1
2,la Barceloneta,79.6,54.43,13.45,32.11,9.19,5165,745.8,Ciutat Vella,41.380653,2.189927,3
3,"Sant Pere, Santa Caterina i la Ribera",99.4,46.91,16.81,36.28,8.03,5152,916.7,Ciutat Vella,41.372251,2.177532,3
4,el Fort Pienc,106.5,62.04,17.92,20.04,6.01,4107,900.4,Eixample,41.395925,2.182325,1
5,la Sagrada Família,101.8,65.28,17.56,17.17,6.08,4209,876.2,Eixample,41.403479,2.174410,1
6,la Dreta de l'Eixample,175.9,64.20,14.85,20.95,4.64,6332,1195.2,Eixample,41.394124,2.166471,3
7,l'Antiga Esquerra de l'Eixample,137.2,66.32,14.25,19.44,5.25,5091,1060.0,Eixample,41.388765,2.156597,3
8,la Nova Esquerra de l'Eixample,110.2,62.19,17.01,20.80,5.83,4465,943.2,Eixample,41.382816,2.149966,1
9,Sant Antoni,104.2,62.85,16.64,20.51,6.36,4591,900.5,Eixample,41.386699,2.163828,1


In [359]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bcn_merged['Latitude'], bcn_merged['Longitude'], bcn_merged['Neighborhood'], bcn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



In [360]:
bcn_merged.loc[bcn_merged['Cluster Labels'] == 0, bcn_merged.columns[[0] + list(range(5, bcn_merged.shape[1]))]]

,Neighborhood,% Unemployment,(€/m²) House prices,Average rental prices (€/month),Borough,Latitude,Longitude,Cluster Labels
12,la Marina de Port,8.73,2723,691.4,Sants-Montjuic,41.360296,2.137584,0
33,Can Baró,6.50,2859,687.0,Horta-Guinardó,41.416966,2.162693,0
36,el Carmel,8.70,2242,603.3,Horta-Guinardó,41.425591,2.154959,0
37,la Teixonera,6.98,2500,648.9,Horta-Guinardó,41.423173,2.145970,0
38,Sant Genís dels Agudells,7.54,2400,674.7,Horta-Guinardó,41.426457,2.137491,0
39,Montbau,7.71,2550,724.8,Horta-Guinardó,41.429550,2.144538,0
40,la Vall d'Hebron,6.93,2600,829.0,Horta-Guinardó,41.424789,2.142317,0
41,la Clota,3.96,2600,651.3,Horta-Guinardó,41.428646,2.153520,0
43,Vilapicina i la Torre Llobeta,8.43,2774,714.8,Nou Barris,41.428207,2.174093,0
44,Porta,8.53,2705,662.0,Nou Barris,41.434898,2.178006,0


In [361]:
bcn_merged.loc[bcn_merged['Cluster Labels'] == 1, bcn_merged.columns[[0] + list(range(5, bcn_merged.shape[1]))]]

,Neighborhood,% Unemployment,(€/m²) House prices,Average rental prices (€/month),Borough,Latitude,Longitude,Cluster Labels
0,el Raval,9.52,4029,748.6,Ciutat Vella,41.379518,2.168368,1
1,el Barri Gòtic,6.97,4884,973.3,Ciutat Vella,41.383395,2.176912,1
4,el Fort Pienc,6.01,4107,900.4,Eixample,41.395925,2.182325,1
5,la Sagrada Família,6.08,4209,876.2,Eixample,41.403479,2.174410,1
8,la Nova Esquerra de l'Eixample,5.83,4465,943.2,Eixample,41.382816,2.149966,1
9,Sant Antoni,6.36,4591,900.5,Eixample,41.386699,2.163828,1
10,el Poble Sec,8.09,3936,749.5,Sants-Montjuic,41.372806,2.161972,1
11,la Marina del Prat Vermell,14.70,4500,813.0,Sants-Montjuic,41.353842,2.142318,1
14,Hostafrancs,6.58,3912,747.1,Sants-Montjuic,41.375088,2.142933,1
18,les Corts,6.20,4821,1002.5,Les Corts,41.385244,2.132863,1


In [362]:
bcn_merged.loc[bcn_merged['Cluster Labels'] == 2, bcn_merged.columns[[0] + list(range(5, bcn_merged.shape[1]))]]

,Neighborhood,% Unemployment,(€/m²) House prices,Average rental prices (€/month),Borough,Latitude,Longitude,Cluster Labels
13,la Font de la Guatlla,8.40,3510,811.1,Sants-Montjuic,41.370782,2.144676,2
15,la Bordeta,7.01,3171,790.0,Sants-Montjuic,41.369142,2.137373,2
16,Sants - Badal,5.68,3469,759.2,Sants-Montjuic,41.375466,2.127710,2
17,Sants,6.31,3666,787.2,Sants-Montjuic,41.375329,2.134912,2
27,Vallcarca i els Penitents,6.43,3541,910.9,Gràcia,41.415712,2.141469,2
29,la Salut,6.44,3561,800.0,Gràcia,41.411866,2.153961,2
32,el Baix Guinardó,6.32,3493,770.9,Horta-Guinardó,41.411441,2.168510,2
34,el Guinardó,7.12,3164,736.9,Horta-Guinardó,41.417733,2.173632,2
35,la Font d'en Fargues,8.93,3206,817.1,Horta-Guinardó,41.424364,2.165320,2
42,Horta,8.85,2970,698.4,Horta-Guinardó,41.433021,2.159088,2


In [363]:
bcn_merged.loc[bcn_merged['Cluster Labels'] == 3, bcn_merged.columns[[0] + list(range(5, bcn_merged.shape[1]))]]

,Neighborhood,% Unemployment,(€/m²) House prices,Average rental prices (€/month),Borough,Latitude,Longitude,Cluster Labels
2,la Barceloneta,9.19,5165,745.8,Ciutat Vella,41.380653,2.189927,3
3,"Sant Pere, Santa Caterina i la Ribera",8.03,5152,916.7,Ciutat Vella,41.372251,2.177532,3
6,la Dreta de l'Eixample,4.64,6332,1195.2,Eixample,41.394124,2.166471,3
7,l'Antiga Esquerra de l'Eixample,5.25,5091,1060.0,Eixample,41.388765,2.156597,3
20,Pedralbes,3.05,6340,1785.6,Les Corts,41.390140,2.112218,3
22,Sarrià,3.09,6042,1353.9,Sarrià - Sant Gervasi,41.399373,2.121513,3
23,les Tres Torres,3.46,5689,1634.1,Sarrià - Sant Gervasi,41.397611,2.131184,3
24,Sant Gervasi - la Bonanova,3.74,5029,1236.7,Sarrià - Sant Gervasi,41.409637,2.129637,3
25,Sant Gervasi - Galvany,3.65,5306,1311.5,Sarrià - Sant Gervasi,41.397807,2.143380,3
66,la Vila Olímpica del Poblenou,6.28,5402,1259.7,Sant Martí,41.389868,2.196846,3


## Results and Discussion <a name="results"></a>

Once we have the two clustered maps of Barcelona we can start to analize the results.Our analysis shows that the most common venues clustering gets 3 big clusters, the 'purple' cluster has the most touristic hoods the most common venues in this hoods are usually restaurants and hotels an they are located in the town center and the left part of the city. The second big cluster is the 'blue-green' cluster, this cluster contains hoods that are not as touristic as the first cluster and the most common venues are for example supermarkets or other shops more oriented in selling to local people. And finally the third big cluster is the 'red' one which contains the hoods of the outskirts of the city and they don't have a lot of venues and are usually different from the town center neighborhoods.

Secondly we have to analize the clustering using socioeconomic data from the hoods of Barcelona. We can see that we have 4 big different clusters. The 'brown' cluster contains the most wealthy hoods in the city (Cluster Label, 3) for example if we take a look at the rent house prices we can see that are much higher than in the other clusters. Also if we take a look at the unemployment rate we can see that is very low in comparison of the rest of the town. The 'purple' cluster we can say that is a medium class cluster (Cluser Label,1) for example if we take a look at the rent house prices we can see that are  higher than other clusters but lower than the 'brown' clusters. Also if we take a look at the unemployment rate we can see that is low or average in comparison of the rest of the town. Finally the 'blue' and 'red' clusters located in the (Clusters Labels,0 and 2) We can see that the unemployment ratings and the rent prices are the opposite of the 'brown' cluster so we can say that they are the poorest hoods in town.



## Conclusion <a name="conclusion"></a>

The main purpose of this project was to identify different shapes of clustering depending on the data used. We can conclude that despite the clusters don't have the same shape with the venues and the socioeconomic data we can conclude that they are quite similar and show us which are the hoods that are more oriented to the tourists and also which are the richest and the poorest hoods and where they are geographically located in the city.

Finally this project helped me to understand better how my city which is Barcelona has different types of hoods and how tourism has affected the city.
